In [13]:
import numpy as np
from amuse.datamodel import Particles
from amuse.units import units
from scipy.integrate import quad

In [16]:
def sample_stellar_mass(sample_imf_mass, num_bins=10, min_samp_mass=1.0,
                              max_samp_mass=150.0, sum_small=False):

    [n_stars, bins, lam, norm] = sample_stars_poisson(sample_imf_mass, min_samp_mass, max_samp_mass, num_bins)

    # Now use that to sample the IMF.
    masses = np.zeros(n_stars.sum())
    k = 0
    for i, n in enumerate(n_stars):
        #print "Pulling ", n, "stars from ranges ", bins[i], "to ", bins[i+1]
        for j in range(n):

            while (masses[k] == 0):

                m = np.random.uniform(low=bins[i], high=bins[i+1])
                r = np.random.uniform()
                p = mkroupa(m, norm)

                if (p/r > 1.0):
                    masses[k] = m

            k+=1

    # Sum all stars < 1 MSun into stars > 1 MSun.
    if (sum_small):
        masses = collect_small_stars_mass(masses)

    np.random.shuffle(masses)

    return masses

def sample_stars_poisson(sink_mass, M_min, M_max, num_bins):
    """
    Return a poisson random sampling from the Kroupa IMF of sink total
    mass from M_min to M_max separated into num_bins in logspace.

    Returns:
        n_stars: Number of stars in each logarithmic bin
        binsL:   The bin edges, including the right most bin edge
        lam:     The average number of stars in each bin that the
                 Poisson sample is centered around.
        norm:    Norm to be used to sample the Kroupa IMF
                 using the n_stars array.
    """

    norm_inv = quad(kroupa,M_min,M_max,args=(1))[0]

    norm = 1/norm_inv

    binsL = np.logspace(np.log10(M_min),np.log10(M_max),num_bins+1)
    mass_per_bin = []
    frac_per_bin = []

    avg_mass = quad(mkroupa, M_min, M_max, args=(norm))[0]

    for i in range(num_bins):
        mass_per_bin.append( quad(mkroupa, binsL[i], binsL[i+1], args=(norm))[0]
                             / quad(kroupa, binsL[i], binsL[i+1], args=(norm))[0] )
        frac_per_bin.append( quad(mkroupa, binsL[i], binsL[i+1], args=(norm))[0]
                             / avg_mass )

    mass_per_bin = np.array(mass_per_bin)
    frac_per_bin = np.array(frac_per_bin)

    lam = sink_mass*frac_per_bin/mass_per_bin

    n_stars = np.random.poisson(lam=lam)

    return n_stars, binsL, lam, norm

def kroupa(m,a):

    if (0.001 <= m < 0.08):
        k = a*m**(-0.3)
    elif (0.08 <= m < 0.5):
        k = a*(0.08)*m**(-1.3)
    elif (0.5 <= m):
        k = a*(0.08*0.5)*m**(-2.3)
    else:
        print("Invalid mass range!")
        k=0
    return k


def mkroupa(m,a):

    if (0.001 <= m < 0.08):
        k = m*a*m**(-0.3)
    elif (0.08 <= m < 0.5):
        k = m*a*(0.08)*m**(-1.3)
    elif (0.5 <= m):
        k = m*a*(0.08*0.5)*m**(-2.3)
    else:
        print("Invalid mass range!")
        k=0
    return k

In [17]:
sample_imf_mass = 10000.0 | units.MSun
sample_imf_bins = 10
max_imf_mass = 150.0 | units.MSun
min_imf_mass = 0.08 | units.MSun
sum_small = False

new_masses = sample_stellar_mass(
                            sample_imf_mass.value_in(units.MSun),
                            num_bins=sample_imf_bins,
                            min_samp_mass=min_imf_mass.value_in(units.MSun),
                            max_samp_mass=max_imf_mass.value_in(units.MSun),
                            sum_small=sum_small,
            )

print(" queued {} stars,".format(len(new_masses)), end='')
print(" mass {},".format(np.sum(new_masses)), end='')
print(" max mass {}".format(np.amax(new_masses)))

 queued 16809 stars, mass 10291.33601566899, max mass 121.86314275212084


In [23]:
high_mass_stars = new_masses > 7.0
print(len(new_masses[high_mass_stars]))
print(np.sort(new_masses[high_mass_stars]))

125
[   7.01710271    7.01769395    7.01781929    7.03165951    7.04692789
    7.05659733    7.06572201    7.14295456    7.1519849     7.17883734
    7.19710843    7.25457339    7.29343468    7.30056172    7.31248224
    7.31499302    7.38650548    7.44384154    7.4520418     7.50618098
    7.59296212    7.63374628    7.67032339    7.7143474     7.77064589
    7.77730221    7.93219975    8.08043557    8.09870626    8.2056488
    8.44008246    8.69268796    8.80118025    8.823361      8.82808139
    8.88953083    9.07882126    9.16822802    9.18108017    9.26452629
    9.39106596    9.54387615    9.63774821    9.65110582    9.75452753
    9.99989364   10.05590185   10.1694597    10.1809412    10.26521675
   10.3963439    10.75353809   10.95786322   10.97205532   11.07037009
   11.0966625    11.32744177   11.39988308   11.52266846   11.67767803
   11.692658     11.71848894   11.76780922   12.1326735    12.21713659
   12.352921     12.4375189    12.46121233   12.6218106    12.93116308
   

In [30]:
# Practicing
pure_rand = np.array([0.1, 0.2, 7.1, 0.4, 1.0, 2.0, 5.0, 0.3, 10.0])

sorted_pure_rand = np.sort(pure_rand)[::-1]
print(sorted_pure_rand)

[ 10.    7.1   5.    2.    1.    0.4   0.3   0.2   0.1]


In [53]:
most_massive = sorted_pure_rand > 7
print(sorted_pure_rand[most_massive])
most_massive_arr = sorted_pure_rand[most_massive]

print(sorted_pure_rand[len(most_massive_arr):])
less_massive_arr = sorted_pure_rand[len(most_massive_arr):]
np.random.shuffle(less_massive_arr)

print(less_massive_arr)

partial_less_massive_arr_rand = less_massive_arr[:3]
print(partial_less_massive_arr_rand)

np.concatenate(most_massive_arr, partial_less_massive_arr_rand)
print(partial_less_massive_arr_rand)

print(sorted_pure_rand)

[ 10.    7.1]
[ 1.   0.2  2.   0.1  5.   0.4  0.3]
[ 0.4  0.2  5.   0.3  1.   2.   0.1]
[ 0.4  0.2  5. ]


TypeError: only integer scalar arrays can be converted to a scalar index